In [10]:
using ArchGDAL
using FreqTables
using LinearAlgebra
using DataFrames
using Dates
using Pipe
using Plots
using Rasters
using Serialization
using SparseArrays

include("util.jl")
include("geo_ana.jl")
include("model_meta_pop.jl")
include("main_model_meta_pop.jl")

validate_meta_population_model

In [11]:
path_spatial = "../dt_tmp/spatial_params_agg230.ser"

"../dt_tmp/spatial_params_agg230.ser"

## International airport location, index 

In [12]:
sp_pars = deserialize(path_spatial)
df = sp_pars.df
# Tambo International
lat = -26.12825796201514
lon = 28.242074092511
dist = harversine_dist.(lat, lon, df[:, :lat], df[:, :lon]) 
println("argmin(dist): $(argmin(dist)), dist: $(dist[argmin(dist)])")

# Cape Town 
lat =  -33.970502228847884
lon = 18.600228711334545
dist = harversine_dist.(lat, lon, df[:, :lat], df[:, :lon]) 
println("argmin(dist): $(argmin(dist)), dist: $(dist[argmin(dist)])")

# King Shaka 
lat =  -29.608764960536764
lon = 31.115368797913593
dist = harversine_dist.(lat, lon, df[:, :lat], df[:, :lon]) 
println("argmin(dist): $(argmin(dist)), dist: $(dist[argmin(dist)])")



argmin(dist): 11, dist: 9.631110668981826
argmin(dist): 7, dist: 5.508420257007044
argmin(dist): 62, dist: 8.80101225388816


# Transmission model 

In [13]:
n_sim = 5000

5000

In [14]:
R0 = 14.0
α = 0.05
pc = 0.25

0.25

In [15]:
imp_ws_moz = CSV.read("../data/imp_ws_moz.csv", DataFrame, header=false)[:,1]
imp_ws_airport = CSV.read("../data/imp_ws_airport.csv", DataFrame, header=false)[:,1]
nothing

In [16]:
include("model_meta_pop.jl")
include("main_model_meta_pop.jl")

validate_meta_population_model

In [17]:
function post_processing(path_trans, path_spatial, pc)
    par_AFP, par_ES = set_par_AFP_ES(path_spatial_params=path_spatial, pc=pc)
    path_save = sensitivity_all_summary(par_AFP, par_ES; path_trans=path_trans)
    validate_meta_population_model(path_trans, path_spatial)
    remove_all_leaving_one(path_trans)
    return path_save
end

function run_all_patterns(;
        R0=1.0, α=0.05, pc=1.0, 
        imp_ws_moz=[1.0], imp_ws_airport=[1.0],
        n_sim=100,
        path_spatial=""
    )
    summary_info = []
    for pattern in ["population_size", "airport", "mozambique"]
        imp_ws = pattern=="airport" ? imp_ws_airport : imp_ws_moz
        path_trans = run_transmission_model(;
            R0=R0, α=α, pc=pc, imp_ws=imp_ws,
            n_sim=n_sim,
            path_spatial_params=path_spatial, 
            pattern=pattern
        )
        path_save = post_processing(path_trans, path_spatial, pc)
        rec = "R0=$(R0), α=$(α), pc=$(pc), n_sim=$(n_sim), pattern=$(pattern), path: $(path_save)"
        push!(summary_info, rec)
    end
    println(summary_info)
end

run_all_patterns (generic function with 1 method)

In [9]:
run_all_patterns(
    R0=R0, α=α, pc=pc, imp_ws_moz=imp_ws_moz, imp_ws_airport=imp_ws_airport, 
    n_sim=n_sim,
    path_spatial=path_spatial,
    )

LoadError: UndefVarError: imp_ws_moz not defined

## Sensitivity analysis 

In [ ]:
summary_info = []
pattern = "population_size"
for R0_sens in [10, 12, 16]
    path_trans = run_transmission_model(;
        R0=R0_sens, α=α, pc=pc, imp_ws=[1.0],
        n_sim=n_sim,
        path_spatial_params=path_spatial, 
        pattern=pattern
    )
    path_save = post_processing(path_trans, path_spatial, pc)
    rec = "R0=$(R0), α=$(α), pc=$(pc), n_sim=$(n_sim), pattern=$(pattern), path: $(path_save)"
    push!(summary_info, rec)
end
println(summary_info)

In [ ]:
summary_info = []
for α_sens in [0.005, 0.010, 0.100, 0.500]
    path_trans = run_transmission_model(;
        R0=R0, α=α_sens, pc=pc, imp_ws=[1.0],
        n_sim=n_sim,
        path_spatial_params=path_spatial, 
        pattern=pattern,
    )
    path_save = post_processing(path_trans, path_spatial, pc)
    rec = "R0=$(R0), α=$(α), pc=$(pc), n_sim=$(n_sim), pattern=$(pattern), path: $(path_save)"
    push!(summary_info, rec)
end
println(summary_info)

## Mozambique risk ordered ES. 

In [18]:
imp_ws_moz_sorted = CSV.read("../data/imp_ws_moz_sorted.csv", DataFrame, header=false)[:,1]
imp_ws_airport_sorted = CSV.read("../data/imp_ws_airport_sorted.csv", DataFrame, header=false)[:,1]
path_spatial_sorted = "../dt_tmp/spatial_params_agg230_moz_sorted.ser"

"../dt_tmp/spatial_params_agg230_moz_sorted.ser"

In [19]:
run_all_patterns(
    R0=R0, α=α, pc=pc, 
    imp_ws_moz=imp_ws_moz_sorted, 
    imp_ws_airport=imp_ws_airport_sorted,
    n_sim=n_sim,
    path_spatial=path_spatial_sorted,
    )

# of sites: 1517
SEIRMetaModelParams
  R0: Float64 14.0
  γ1: Float64 0.25
  γ2: Float64 0.06657789613848203
  σ: Float64 0.3291
  P_AFP: Float64 0.005
  N_tot: Array{Int64}((1517,)) [16942, 6206, 11638, 17796, 9978, 18076, 14578, 18809, 13780, 6951  …  115, 107, 107, 110, 101, 105, 102, 105, 101, 103]
  N_unvac: Array{Int64}((1517,)) [2049, 750, 1407, 2152, 1206, 2186, 1763, 2274, 1666, 840  …  7, 2, 6, 6, 7, 6, 7, 6, 6, 6]
  I0_init: Int64 1
  I0_ind: Int64 1
  days: Int64 1095
  n_site: Int64 1517
  α: Float64 0.05
  π_mat: Array{Float64}((1517, 1517)) [0.0 0.09700668215283446 … 6.40008051061156e-8 6.351348570063297e-8; 0.40108404561682365 0.0 … 2.3434779415237e-8 2.3265534898956926e-8; … ; 5.945035894425645e-8 2.2301115523520442e-8 … 0.0 3.73407299880895e-8; 6.393101238222124e-8 2.4169055529270278e-8 … 3.661928733425369e-8 0.0]
  μ: Float64 0.000547945205479452
  β: Float64 0.9320905459387484
  pc: Float64 0.25
  imp_ws: Array{Float64}((1517,)) [0.06333313266942489, 0.0399490000078

Progress: 100%|█████████████████████████████████████████| Time: 2:31:32


7114.235295 seconds (58.33 M allocations: 16.414 TiB, 2.01% gc time, 0.01% compilation time)
Cum index:107
Actual ES coverage: 34.291977907735735
AFPSurParams
  P_test: Float64 0.97
  P_sample: Float64 0.53
  P_H: Float64 0.9
ESParams
  g: Float64 0.23025850929940458
  P_test: Float64 0.97
  area: Array{Bool}((1517,)) Bool[1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  n_freq: Int64 30


Progress: 100%|█████████████████████████████████████████| Time: 0:22:59


../dt_tmp_res/20231117_111513.ser


...

11095.856094 seconds (86.82 M allocations: 27.104 TiB, 0.94% gc time)
Cum index:107
Actual ES coverage: 34.291977907735735
AFPSurParams
  P_test: Float64 0.97
  P_sample: Float64 0.53
  P_H: Float64 0.9
ESParams
  g: Float64 0.23025850929940458
  P_test: Float64 0.97
  area: Array{Bool}((1517,)) Bool[1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  n_freq: Int64 30


Progress: 100%|█████████████████████████████████████████| Time: 0:21:36


../dt_tmp_res/20231117_162723.ser


Progress: 100%|█████████████████████████████████████████| Time: 0:06:05


Any["R0=14.0, α=0.05, pc=0.25, n_sim=5000, pattern=population_size, path: ../dt_tmp_res/20231117_111513.ser", "R0=14.0, α=0.05, pc=0.25, n_sim=5000, pattern=airport, path: ../dt_tmp_res/20231117_125438.ser", "R0=14.0, α=0.05, pc=0.25, n_sim=5000, pattern=mozambique, path: ../dt_tmp_res/20231117_162723.ser"]


## Debug field

In [ ]:
path_trans = run_transmission_model(;
    R0=R0, α=α, pc=pc, imp_ws=imp_ws,
    n_sim=n_sim,
    path_spatial_params=path_spatial, 
    pattern="population_size"
)
par_AFP, par_ES = set_par_AFP_ES(path_spatial_params=path_spatial, pc=pc)
#baseline_results(par_AFP, par_ES; 
#    path_trans=path_trans
#)

sensitivity_all_summary(par_AFP, par_ES; path_trans=path_trans)
validate_meta_population_model(path_trans, path_spatial)
remove_all_leaving_one(path_trans)

In [ ]:
path_trans = run_transmission_model(;
    R0=R0, α=α, pc=pc, n_sim=n_sim,
    path_spatial_params=path_spatial, 
    pattern="airport"
)
par_AFP, par_ES = set_par_AFP_ES(path_spatial_params=path_spatial)
sensitivity_all_summary(par_AFP, par_ES; path_trans=path_trans)
validate_meta_population_model(path_trans, path_spatial)
remove_all_leaving_one(path_trans)

In [ ]:
path_trans = run_transmission_model(;
    R0=R0, α=α, pc=pc, imp_ws=imp_ws,
    n_sim=n_sim,
    path_spatial_params=path_spatial, 
    pattern="mozambique"
)
par_AFP, par_ES = set_par_AFP_ES(path_spatial_params=path_spatial)
sensitivity_all_summary(par_AFP, par_ES; path_trans=path_trans)
validate_meta_population_model(path_trans, path_spatial)
remove_all_leaving_one(path_trans)